[大图](TF1.html), [下载](origin_files/TF1.ipynb)

In [1]:
import tensorflow as tf

In [1]:
hello=tf.constant('hello!')
sess=tf.Session()#创建一个session
hey=sess.run(hello)#使用session来运行
# sess.close()#关闭session，释放资源

In [2]:
tf.contrib.slimim

<module 'tensorflow.contrib.slim' from 'C:\\Users\\guofei\\Anaconda3\\lib\\site-packages\\tensorflow\\contrib\\slim\\__init__.py'>

# Session

In [2]:
a=tf.constant([10.1,2.0],name='a')
b=tf.constant([1.0,2.1],name='b')
sess.run(a+b)

array([ 11.10000038,   4.0999999 ], dtype=float32)

In [3]:
with tf.Session() as sess:
    sess.run(a+b)
# 无需调用sess.close()，而是运行完毕后自动释放资源

In [4]:
sess=tf.Session()
result=a+b
with sess.as_default():
    print(result.eval())

[ 11.10000038   4.0999999 ]


In [5]:
#等价于：
sess=tf.Session()
result.eval(session=sess)

array([ 11.10000038,   4.0999999 ], dtype=float32)

## InteractiveSession
注册为默认Session，以后的运算会默认在这个session中运行

In [6]:
import tensorflow as tf
sess=tf.InteractiveSession()

## ConfigProto

In [7]:
config=tf.ConfigProto(allow_soft_placement=True, # 优先使用GPU，GPU不支持的时候换成CPU，而不是报错
                     log_device_placement=True) #记录节点分配日志，生产环境下可以设定为False以减小日志量

In [8]:
sess1=tf.InteractiveSession(config=config)
sess2=tf.Session(config=config)

# 变量

In [9]:
# a=tf.matmul(x,w1)#矩阵乘法

In [10]:
import math
# 参数初始化
# 均值0，方差1的正态随机数：
weights=tf.Variable(tf.truncated_normal([3,2],stddev=1),name='weights')
#正态随机数除以神经元的个数
weights=tf.Variable(tf.truncated_normal([500,25],stddev=1/math.sqrt(500)),name='weights')

In [11]:
tf.random_normal #正态分布
tf.truncated_normal #正态分布，如果离差超过2标准差，则重新随机
tf.random_uniform#均匀分布
tf.random_gamma#Gamma分布

# 常数：
tf.zeros(shape=[1,2])
tf.ones
tf.fill(dims=(2,3),value=9)#生成一个2*3矩阵，并用9填充
tf.constant([1,2,4])

<tf.Tensor 'Const_1:0' shape=(3,) dtype=int32>

### 设定初始值

In [12]:
biases=tf.Variable(tf.zeros([3]))
w2=tf.Variable(weights.initialized_value())
w3=tf.Variable(2*weights.initialized_value())

- tf.Variable定义的tensor，一定要sess.run(w2.initializer)之后才可以参与运算

In [13]:
sess.run(w2.initializer)
sess.run(w3.initializer)
sess.run(w2+w3)

array([[  1.40550081e-02,   3.56171094e-02,   8.82129595e-02, ...,
         -9.59602743e-02,  -2.08483525e-02,   8.82780626e-02],
       [ -3.08074057e-04,  -6.06677905e-02,   5.50433137e-02, ...,
          1.12989634e-01,   3.39948237e-02,   7.19436184e-02],
       [ -3.74090523e-02,   3.51328403e-02,   1.85388178e-01, ...,
         -7.59455636e-02,   1.36960939e-01,   1.39769107e-01],
       ..., 
       [ -6.68679848e-02,   1.59125775e-04,  -1.43939406e-01, ...,
         -6.22220933e-02,   3.61499526e-02,  -1.48586512e-01],
       [  4.02176231e-02,  -8.69320333e-02,   4.66071218e-02, ...,
          9.19466093e-02,   1.90996770e-02,  -7.27092624e-02],
       [ -1.10123560e-01,   1.02122873e-01,   6.27010167e-02, ...,
          1.09118499e-01,   1.31105542e-01,   1.89743191e-03]], dtype=float32)

- 也可以用global_variables_initializer()初始化所有变量

In [14]:
w4=tf.Variable(weights.initialized_value())
w5=tf.Variable(weights.initialized_value())
tf.global_variables_initializer().run(session=sess)
sess.run(w4+w5)

array([[ 0.01311258, -0.15321772, -0.03100876, ...,  0.01576635,
        -0.17011122,  0.00778334],
       [-0.00632112, -0.0805592 , -0.13366885, ...,  0.05962479,
         0.00965144,  0.04564803],
       [ 0.16788766, -0.06252999,  0.1152531 , ..., -0.13965486,
        -0.0107923 ,  0.16111413],
       ..., 
       [-0.0633782 ,  0.06003167, -0.04054847, ..., -0.05182007,
         0.05266857, -0.05856436],
       [-0.04858895,  0.00196341,  0.02902997, ...,  0.07324979,
        -0.11008207, -0.06104498],
       [ 0.02623019,  0.08211529,  0.14126727, ..., -0.06122007,
         0.00826732, -0.01249944]], dtype=float32)

# 一个信号传播的例子

In [15]:
import tensorflow as tf
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

# 输入信号
x=tf.constant([[0.7,0.9]])

#矩阵积
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)

sess=tf.Session()
sess.run(w1.initializer)#初始化
sess.run(w2.initializer)
sess.run(y)



array([[ 3.95757794]], dtype=float32)

## placeholder占位
定义一个位置，这个位置的数据在运行时才指定，可以节省常量。

In [16]:
import tensorflow as tf
w1=tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2=tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

# 输入信号
x=tf.placeholder(tf.float32,shape=(1,2),name='input')

#矩阵积
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)

sess=tf.Session()
sess.run(w1.initializer)#初始化
sess.run(w2.initializer)
sess.run(y,feed_dict={x:[[0.7,0.9]]})

array([[ 3.95757794]], dtype=float32)

# iris为例的一个完整的神经网络训练过程

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
sess=tf.Session()

- 加载数据并dummy

In [2]:
from sklearn import datasets
dataset=datasets.load_iris()
data=dataset.data
target=dataset.target
X=data
target_df=pd.Series(target)
target_df=pd.get_dummies(target_df)
target=target_df.values
from sklearn import model_selection
train_data,test_data,train_target,test_target=model_selection.train_test_split(
    data,target,test_size=0.2,train_size=0.8,random_state=12345)#划分训练集和测试集

- 基本参数

In [3]:
X=train_data
Y=train_target
dataset_size=data.shape[0]
batch_size=10

- 定义张量

In [4]:
w1=tf.Variable(tf.truncated_normal(shape=(4,10),stddev=1),name='weight1')
b1=tf.Variable(tf.truncated_normal(shape=(1,10),stddev=1),name='bias1')
w2=tf.Variable(tf.truncated_normal(shape=(10,3),stddev=1),name='weight2')
b2=tf.Variable(tf.truncated_normal(shape=(1,3),stddev=1),name='bias2')
x=tf.placeholder(shape=(None,4),dtype=tf.float32)
y_=tf.placeholder(shape=(None,3),dtype=tf.float32)

- 信号正向传播

In [5]:
a=tf.nn.sigmoid(tf.matmul(x,w1)+b1)
y=tf.nn.softmax(tf.matmul(a,w2)+b2)

- 误差与优化

In [6]:
cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cross_entropy)

- 迭代学习

In [7]:
sess.run(tf.global_variables_initializer())#全部初始化

STEPS=50000
for i in range(STEPS):
    start=(i*batch_size)%batch_size
    end=min(start+batch_size,dataset_size)
    sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
    if i%5000==0:
        total_cross_entropy=sess.run(cross_entropy,feed_dict={x:X,y_:Y})
        print(i,'training step,cross entropy is',total_cross_entropy)

0 training step,cross entropy is 1.7206
5000 training step,cross entropy is 0.744731
10000 training step,cross entropy is 0.577951
15000 training step,cross entropy is 0.49825
20000 training step,cross entropy is 0.435902
25000 training step,cross entropy is 0.380805
30000 training step,cross entropy is 0.33211
35000 training step,cross entropy is 0.291082
40000 training step,cross entropy is 0.257882
45000 training step,cross entropy is 0.231364


- predict

In [8]:
y_train_predict=sess.run(y,feed_dict={x:train_data})
y_test_predict=sess.run(y,feed_dict={x:test_data})
# sess.close()

- 模型评价

In [9]:
import sklearn.metrics as metrics
print('train',metrics.confusion_matrix(train_target.argmax(axis=1),y_train_predict.argmax(axis=1)))
print('test',metrics.confusion_matrix(test_target.argmax(axis=1),y_test_predict.argmax(axis=1)))

train [[39  0  0]
 [ 0 34  4]
 [ 0  0 43]]
test [[11  0  0]
 [ 0 10  2]
 [ 0  0  7]]


In [10]:
correct_prediction=tf.equal(tf.argmax(y,axis=1),tf.argmax(y_,axis=1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
sess.run(accuracy,feed_dict={x:train_data,y_:train_target})

0.96666664

In [ ]:
tf.GraphKeys.GLOBAL_VARIABLES #所有变量
tf.GraphKeys.TRAINABLE_VARIABLES #所有可训练变量
tf.GraphKeys.SUMMARIES #日志相关张量
tf.GraphKeys.QUEUE_RUNNERS #处理输入的QueueRunner
tf.GraphKeys.MOVING_AVERAGE_VARIABLES #所有计算了移动平均值的变量

# 其它代码

In [ ]:
with tf.Graph().as_default():
    pass


In [ ]:
import math
# 参数初始化
# 均值0，方差1的正态随机数：
weights=tf.Variable(tf.truncated_normal([3,2],stddev=1),name='weights')
#正态随机数除以神经元的个数
weights=tf.Variable(tf.truncated_normal([500,25],stddev=1/math.sqrt(500)),name='weights')

In [ ]:
# DropOut
# 每一轮训练临时性的关闭一些节点
# 降低VC维数量，减少过拟合风险

keep_prob=tf.placeholder(tf.float32)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)

# slim
很有用的东西，可以不用设置中间变量，给函数赋值

In [6]:
slim=tf.contrib.slim
with slim.arg_scope([slim.conv2d,slim.fully_connected],weights_regularizer=slim.l2_regularizer(weights_decay)):
    pass
